In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2648).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes1307.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../../best_model2306/pytorch_model10d.bin", map_location=device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [2]:
#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("../data/desc.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [3]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = dict()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result[id] = dscr[dscr['id']==id[:4]].iloc[0]['label']
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

In [4]:
#Пример с описанием
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')

{'1704906500': 'Кондитерские изделия из сахара (включая белый шоколад), не содержащие какао'}

In [5]:
#Пример без описания
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО', desc=False)

'1704906500'

In [6]:
#функция предикта классов с вероятностью (софтмакс), по умолчанию 5 классов
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [12]:
#Пример с 3 классами
print(predict_prob_with_descr('трубы сантехнические пластиковые ', 3))

[['3917390001', 'Трубы, трубки, шланги и их фитинги (например, соединения, колена, фланцы), из пластмасс', 0.22377032041549683], ['3917390008', 'Трубы, трубки, шланги и их фитинги (например, соединения, колена, фланцы), из пластмасс', 0.1157531589269638], ['3917330009', 'Трубы, трубки, шланги и их фитинги (например, соединения, колена, фланцы), из пластмасс', 0.08061529695987701]]


In [11]:
#Пример с 5 классами
print(predict_prob_with_descr('трубы металлические ', 5))

[['7304900009', 'Трубы, трубки и профили полые, бесшовные, из черных металлов (кроме чугунного литья)', 0.1668553650379181], ['8307100009', 'Трубы гибкие из недрагоценных металлов, с фитингами или без них', 0.15347743034362793], ['7304499309', 'Трубы, трубки и профили полые, бесшовные, из черных металлов (кроме чугунного литья)', 0.08873266726732254], ['7306699009', 'Трубы, трубки и профили полые прочие (например, с открытым швом или сварные, клепаные или соединенные аналогичным способом), из черных металлов', 0.059598829597234726], ['7304312008', 'Трубы, трубки и профили полые, бесшовные, из черных металлов (кроме чугунного литья)', 0.043118175119161606]]


In [10]:
#функция предикта классов и описания с вероятностью, по умолчанию 5 шт. Использует функцию predict_prob.
#возвращает список списков
def predict_prob_with_descr(text, qtty=5):
    probs = predict_prob(text, qtty=qtty)
    #result = np.array()
    result = list()
    for each in probs:
        result.append([each, dscr[dscr['id']==each[:4]].iloc[0]['label'], probs[each]])
    return result

In [13]:
#Пример с 5 классами.
predict_prob_with_descr('колеса автомобильные')

[['8714103000',
  'Части и принадлежности к транспортным средствам товарных позиций 8711 - 8713',
  0.3244084417819977],
 ['8708709909',
  'Части и принадлежности моторных транспортных средств товарных позиций 8701 - 8705',
  0.1690853089094162],
 ['8708705009',
  'Части и принадлежности моторных транспортных средств товарных позиций 8701 - 8705',
  0.06727831065654755],
 ['4011900000',
  'Шины и покрышки пневматические резиновые новые',
  0.02651013806462288],
 ['8714999009',
  'Части и принадлежности к транспортным средствам товарных позиций 8711 - 8713',
  0.01494534034281969]]

In [14]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('орехи грецкие'), columns=['ID', 'LABEL', 'PROB'])

,ID,LABEL,PROB
0,0802310000,"Прочие орехи, свежие или сушеные, очищенные от...",0.865241
1,0802320000,"Прочие орехи, свежие или сушеные, очищенные от...",0.096649
2,0910999900,"Имбирь, шафран, турмерик (куркума), тимьян, ил...",0.000948
3,0711590000,[с 01.01.22]Овощи консервированные для кратков...,0.000633
4,1704909800,Кондитерские изделия из сахара (включая белый ...,0.000536


In [15]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('кофе арабика'), columns=['ID', 'LABEL', 'PROB'])

,ID,LABEL,PROB
0,0901210001,"Кофе, жареный или нежареный, с кофеином или бе...",0.413377
1,2101129201,"Экстракты, эссенции и концентраты кофе, чая ил...",0.255713
2,0901210009,"Кофе, жареный или нежареный, с кофеином или бе...",0.133681
3,0901220009,"Кофе, жареный или нежареный, с кофеином или бе...",0.019602
4,1704909800,Кондитерские изделия из сахара (включая белый ...,0.005251


In [34]:
def consolelog(probs):
    print('prob-ty', 'Code', 'Description', sep=':    ')
    #print('____________________________________________________________________')
    for each in probs:
        if len(each[1]) < 40:
            des = each[1]
        else:
            des = each[1][:40] + '...'
        print(f"{each[2]:.{5}f}", each[0], des, sep=':    ')
        #print('____________________________________________________________________')

In [39]:
consolelog(predict_prob_with_descr("аттракционы", 5))

prob-ty:    Code:    Description
0.00686:    8714993000:    Части и принадлежности к транспортным ср...
0.00610:    8541401000:    [с 01.01.22] Приборы полупроводниковые (...
0.00571:    9508900000:    [с 01.01.22] Цирки передвижные и зверинц...
0.00551:    8529906509:    Части, предназначенные исключительно или...
0.00536:    9508260000:    [с 01.01.22] Цирки передвижные и зверинц...


In [16]:
#загрузим обучающий датасет
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#df = pd.read_csv("../data/firstset.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [43]:
#пример использования по полному коду (можно делать по части кода)
whats_data('9508260000')

,id,label
196551,9508260000,аттракционы парков развлечений и аттракционы в...
197546,9508260000,аттракционы парков развлечений и аттракционы в...


In [41]:
#еще пример
df[df['id'].isin(list(whats_data('0901')['id'].unique()))]

,id,label
72279,0901210009,кофе нат жареный молотый черная карта премиум ...
72280,0901210009,кофе натуральный жареный молотый s caffe deliz...
72281,0901210009,кофе жареный с кофеином молотый в вакуумной по...
72282,0901210009,кофе жареный молотый порошкообразный с кофеино...
72283,0901210009,жареный кофе с кофеином натуральный жареный мо...
...,...,...
133801,0901210009,кофе молотый обжаренный в мешке 1 шт и пачках ...
134050,0901210009,кофе молотый обжаренный новый
134051,0901220009,образцы для проведения работ по оценке соответ...
134444,0901210001,кофе зерновой обжаренный новый


In [40]:
print(predict_class("Велосипед"), predict_class("велосипед"), predict_class("Велосипедов"))

{'8712003000': 'Велосипеды двухколесные и прочие велосипеды (включая трехколесные велосипеды для доставки грузов) без двигателя'} {'8712003000': 'Велосипеды двухколесные и прочие велосипеды (включая трехколесные велосипеды для доставки грузов) без двигателя'} {'8712003000': 'Велосипеды двухколесные и прочие велосипеды (включая трехколесные велосипеды для доставки грузов) без двигателя'}
